In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 3.2MB/s 


In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
import eli5
from eli5.sklearn import PermutationImportance
from ast import literal_eval
from tqdm import tqdm_notebook

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [3]:
cd "/content/drive/My Drive/Colab Notebooks/data_workshop_matrix/Matrix I"

/content/drive/My Drive/Colab Notebooks/data_workshop_matrix/Matrix I


In [4]:
ls

 data/  'Matrix I Day 3.ipynb'  'Matrix I Day 4.ipynb'  'Matrix I Day 5.ipynb'


In [0]:
df = pd.read_csv('data/men shoes.csv', low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X= df[feats].values
  y =df['prices_amountmin'].values
  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
df['brand_cat'] = df['brand'].factorize()[0]

In [8]:
run_model(['brand_cat'])

(-58.38655694633361, 4.223555478221712)

In [9]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=8888)
run_model(['brand_cat'], model)

(-57.44691461889829, 4.247915220032193)

In [10]:
df.head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,reviews,sizes,skus,sourceurls,upc,vin,websiteids,weight,brand_cat
0,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",39.89,39.89,NaN,NaN,NaN,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,True,NaN,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
1,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",51.99,51.99,NaN,NaN,new,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,False,UnbeatableSale - Walmart.com,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
2,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,40.02,40.02,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2016-03-08T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
3,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,50.31,50.31,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2015-11-30T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
4,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""

In [0]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]

In [12]:
run_model(['brand_cat'], model)

(-57.33281703110057, 4.181924526920066)

In [13]:
df['features'].head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [14]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'
literal_eval(str_dict)

[{'key': 'Gender', 'value': ['Men']},
 {'key': 'Shoe Size', 'value': ['M']},
 {'key': 'Shoe Category', 'value': ["Men's Shoes"]},
 {'key': 'Color', 'value': ['Multicolor']},
 {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']},
 {'key': 'Brand', 'value': ['Josmo']}]

In [15]:
literal_eval(str_dict)[0]['value'][0]

'Men'

In [0]:
def parse_features(x):
  output_dict={}
  if str(x) == 'nan': return output_dict
  features = literal_eval(x.replace('\\"','"'))

  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key]=value

  return output_dict
  
df['features_parsed'] = df['features'].map(parse_features)

In [17]:
df['features_parsed'].head()[0]

{'brand': 'josmo',
 'color': 'multicolor',
 'gender': 'men',
 'manufacturer part number': '8190-w-navy-7.5',
 'shoe category': "men's shoes",
 'shoe size': 'm'}

In [18]:
keys = set()
df['features_parsed'].map(lambda x: keys.update(x.keys()))

0        None
1        None
2        None
3        None
4        None
         ... 
18275    None
18276    None
18277    None
18278    None
18279    None
Name: features_parsed, Length: 18280, dtype: object

In [19]:
len(keys)

476

In [0]:
def get_name_feat(key):
  return 'feat'+key

In [21]:
for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df['features_parsed'].map(lambda feats: feats[key] if key in feats else np.nan)

In [22]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'featpower type', 'featlabel', 'featcase material',
       'featshipping weight', 'featmotion control',
       'featsafety apparel product type', 'featincludes', 'featwhat includes',
       'featwatch case shape', 'featorigin of components'],
      dtype='object', length=526)

In [23]:
df[False == df['featathlete'].isnull()].shape[0]/df.shape[0]*100

0.0437636761487965

In [0]:
keys_stat ={}
for key in keys:
  keys_stat[key] = df[False == df[get_name_feat(key)].isnull()].shape[0]/df.shape[0]*100

In [25]:
keys_stat.items()

dict_items([('watch shape', 0.016411378555798686), ('place of origin', 0.10940919037199125), ('frame material:', 0.5142231947483589), ('display', 0.49781181619256015), ('number of heat settings', 0.02188183807439825), ('outer material', 0.010940919037199124), ('finish', 0.26805251641137856), ('removable liner', 0.05470459518599562), ('case thickness', 0.30087527352297594), ('wind resistant', 0.05470459518599562), ('use', 0.010940919037199124), ('ul safety listing', 0.05470459518599562), ('country of manufacture', 0.2735229759299781), ('leather :', 0.010940919037199124), ('seller inventory number', 0.005470459518599562), ('type 2', 0.016411378555798686), ('bridge/temple size:', 0.5142231947483589), ('to fit', 0.038293216630196934), ('clothing category', 0.07658643326039387), ('mirrored', 0.005470459518599562), ('digital camera', 0.05470459518599562), ('chain included', 0.010940919037199124), ('sleeve length', 0.700218818380744), ('date first available at amazon.co.uk', 0.043763676148796

In [26]:
{k:v for k,v in keys_stat.items() if v > 30 }

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['featbrand_cat'] = df['featbrand'].factorize()[0]


In [0]:
df['featcolor_cat'] = df['featcolor'].factorize()[0]
df['featgender_cat'] = df['featgender'].factorize()[0]
df['featmanufacturer part number_cat'] = df['featmanufacturer part number'].factorize()[0]
df['featmaterial_cat'] = df['featmaterial'].factorize()[0]

df['featsport_cat'] = df['featsport'].factorize()[0]
df['featstyle_cat'] = df['featstyle'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]
  

In [29]:
df[df['brand'] == df['featbrand']].shape

(52, 531)

In [0]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())

In [0]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=8888)

In [32]:
run_model(['brand_cat'], model)

(-57.33281703110057, 4.181924526920066)

In [0]:
feats = ['brand_cat','featbrand_cat','featcolor_cat','featgender_cat','featmaterial_cat', 'featshape_cat', 'featoccasion_cat']

In [96]:
run_model(feats, model)
results = run_model(feats, model)
results

(-57.01365238155857, 4.3654235211612065)

In [97]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=8888)
X= df[feats].values
y =df['prices_amountmin'].values
model.fit(X,y)

print(results)
perm = PermutationImportance(model, random_state=8888).fit(X,y)
eli5.show_weights(perm, feature_names=feats)

(-57.01365238155857, 4.3654235211612065)


Weight,Feature
0.2582 ± 0.0120,brand_cat
0.1031 ± 0.0096,featmaterial_cat
0.0244 ± 0.0015,featgender_cat
0.0225 ± 0.0017,featbrand_cat
0.0134 ± 0.0003,featshape_cat
0.0077 ± 0.0017,featcolor_cat
0.0042 ± 0.0011,featoccasion_cat
0.0009 ± 0.0002,weight_cat


In [38]:
df['brand'].value_counts(normalize=True)

nike                   0.097210
puma                   0.033315
ralph lauren           0.028775
vans                   0.021116
new balance            0.020295
                         ...   
dan post boots         0.000055
o'neal                 0.000055
silhouette             0.000055
nba                    0.000055
diamondback fitness    0.000055
Name: brand, Length: 1732, dtype: float64

In [0]:
df_nike = df[df['brand']== 'nike']

In [42]:
df_nike['features_parsed'].head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [48]:
df_nike['features_parsed'].sample(5).values

array([{'manufacturer part number': '616575-370', 'shoe size': '11.5 d(m) us', 'gender': 'men', 'shoe category': 'mens shoes', 'brand': 'nike', 'color': 'night factor/varsity maize'},
       {'style': 'windbreaker', 'material': 'polyester', 'country/region of manufacture': 'china', 'condition': 'new with tags'},
       {'sport': 'soccer', 'main color': 'multi-colored'},
       {'gender': 'men', 'shoe category': 'mens shoes', 'color': 'black/gym red', 'casual & dress shoe style': 'athletic sneakers', 'manufacturer part number': '724734006', 'brand': 'nike', 'shoe width': 'medium (d, m)'},
       {'sport': 'soccer cleats', 'occasion': 'soccer cleats', 'material': 'synthetic-and-leather', 'gender': 'men', 'shoe size': '8 us men', 'shoe category': "men's shoes", 'assembled product dimensions (l x w x h)': '12.00 x 8.00 x 6.00 inches', 'fabric content': 'synthetic-and-leather', 'color': 'black', 'model': '631517 330', 'shoe closure': 'lace-up', 'casual & dress shoe style': 'soccer shoes', '

In [0]:
for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]

In [0]:
feats_cat = [x for x in df.columns if '_cat' in x and x != 'feat_catalog']
feats += feats_cat
feats = list(set(feats))

In [62]:
df['weight'].unique()

array([nan, '3.0 lbs', '9 g', '1.45 lbs', '0.45 lbs', '1.0 lbs',
       '0.23 lbs', '5.0 lbs', '5.5 lbs', '7.45 lbs', '4.0 lbs',
       '2.7969 lbs', '3.9 lbs', '4.6 pounds', '2.1 lbs', '1.1057 lbs',
       '15.0 lbs', '2.4 ounces', '454 g', '0.105 lbs', '9.1 ounces',
       '4.8 lbs', '6.1 lbs', '6.5 lbs', '1.1041 lbs', '1.3 Kg', '91 g',
       '20.0 lbs', '6.0 lbs', '386 g', '0.81 lbs', '4.5 lbs',
       '0.5 ounces', '2.0 lbs', '3.13 lbs', '5.9 lbs', '6.15 lbs',
       '1 pounds', '1.95 lbs', '2.15 lbs', '2 pounds', '2.1 pounds',
       '14 Kg', '0.4788 lbs', '10.0 lbs', '0.38 lbs', '2.5 lbs',
       '68.912 lbs', '45 g', '13.09 lbs', '2.5 pounds', '0.21 lbs',
       '16.75 lbs', '6.3 lbs', '272 g', '1.8 Kg', '2.8 pounds', '0.1 lbs',
       '5.05 lbs', '0.28 lbs', '76.08 lbs', '0.15 lbs', '200 g',
       '7.8 pounds', '399 g', '4.95 lbs', '64.144 lbs', '24 pounds',
       '73.696 lbs', '1.6 lbs', '6.6 ounces', '5 g', '1.2 Kg', '862 g',
       '3.05 lb', '8.6 ounces', '3.6 lbs', '71.

In [0]:
df['weight'] = df['weight'].astype(str)

In [0]:
def convert_weight_to_g(weight):
  if 'nan' in weight:
    return '0'
  elif 'lbs' in weight:
    return float(weight[0:-4])*453.92
  elif 'pounds' in weight:
    return float(weight[0:-7])*453.92
  elif 'ounces' in weight:
    return float(weight[0:-7])*28.35
  elif 'Kg' in weight:
    return float(weight[0:-3])*1000
  elif 'g' in weight:
    return weight[0:-2]
  

In [0]:
df['weight_converted_to_g'] = df['weight'].map(lambda x: str(convert_weight_to_g(x)))

In [0]:
df['weight_cat'] = df['weight_converted_to_g'].factorize()[0]